In [1]:
import numpy as np
import pprint
import ray
ray.init()
checkpoint_path = "model_checkpoint/best_model"

2023-04-07 17:47:14,593	INFO worker.py:1553 -- Started a local Ray instance.


In [4]:
from multiagent_env import GridWorldEnv
from ray.rllib.algorithms.impala import ImpalaConfig, ImpalaTF1Policy, ImpalaTF2Policy, ImpalaTorchPolicy
config = ImpalaConfig()
config = config.training(lr=0.0003, train_batch_size=512)  
config = config.resources(num_gpus=1)
config = config.framework("torch")  
# config = config.rollouts(num_rollout_workers=8)  
print(config.to_dict())  

config = config.environment(env=GridWorldEnv)
config = config.rollouts(num_rollout_workers=8)
# config["exploration_config"] = {
#     "type": "UCB",
#     "ucb_coeff": 2.0
# }

env = GridWorldEnv()
policies = {
    "policy_1": (ImpalaTF2Policy, env.observation_space, env.action_space, {}),
}
policy_mapping_fn = lambda agent_id, episode, worker, **kwargs: "policy_1"

rllib_trainer = config.build()

{'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'num_gpus': 1, 'num_cpus_per_worker': 1, 'num_gpus_per_worker': 0, '_fake_gpus': False, 'num_trainer_workers': 0, 'num_gpus_per_trainer_worker': 0, 'num_cpus_per_trainer_worker': 1, 'custom_resources_per_worker': {}, 'placement_strategy': 'PACK', 'eager_tracing': False, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU': 1}, 'allow_soft_placement': True}, 'local_tf_session_args': {'intra_op_parallelism_threads': 8, 'inter_op_parallelism_threads': 8}, 'env': None, 'env_config': {}, 'observation_space': None, 'action_space': None, 'env_task_fn': None, 'render_env': False, 'clip_rewards': None, 'normalize_actions': True, 'clip_actions': False, 'disable_env_checking': False, 'is_atari': None, 'auto_wrap_old_gym_envs': True, 'num_envs_per_worker': 1, 'samp

(RolloutWorker pid=1512295) 2023-04-07 17:47:54,921	WARNING env.py:296 -- Your MultiAgentEnv <GridWorldEnv instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.


In [5]:
checkpoint_path = "model_checkpoint/best_model"
best_reward = 0
for _ in range(1000):
    results = rllib_trainer.train()
    print(f"Iteration={rllib_trainer.iteration}: R(\"return\")={results['episode_reward_mean']}")
    if results['episode_reward_mean'] > best_reward:
        best_reward = results['episode_reward_mean']
        rllib_trainer.save(checkpoint_path)
    if results['episode_reward_mean'] > 1000:
        break

Iteration=1: R("return")=269.8414634146341
Iteration=2: R("return")=115.78947368421052
Iteration=3: R("return")=-233.5664739884393
Iteration=4: R("return")=-270.49397590361446
Iteration=5: R("return")=-273.79761904761904
Iteration=6: R("return")=-179.32738095238096
Iteration=7: R("return")=-438.38235294117646
Iteration=8: R("return")=-434.03614457831327
Iteration=9: R("return")=-260.07100591715977
Iteration=10: R("return")=-247.04117647058823
Iteration=11: R("return")=-222.22222222222223
Iteration=12: R("return")=-159.66265060240963
Iteration=13: R("return")=-141.92261904761904
Iteration=14: R("return")=-164.19277108433735
Iteration=15: R("return")=-158.25766871165644
Iteration=16: R("return")=-128.16666666666666
Iteration=17: R("return")=-164.87951807228916
Iteration=18: R("return")=-133.52409638554218
Iteration=19: R("return")=-116.0
Iteration=20: R("return")=-132.8470588235294
Iteration=21: R("return")=-158.3012048192771
Iteration=22: R("return")=-192.67073170731706
Iteration=23: R(